# Taller 1. 

## 2. Cargar red GOT

In [1]:
#Carga librerías importantes
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import operator
import pip
import csv
import pickle
import operator

#Read GOT network

#Create empty network

GOT= nx.DiGraph()
#read nodes
nodes= csv.reader(open('./Nodes_GOT(1).csv'))
next(nodes,None)

for row in nodes:
    GOT.add_node(row[0])  
    GOT.node[row[0]]['House']=row[3]

    
#read edges
edges=csv.reader(open('./Edges_GOT(1).csv'))

next(edges,None)

for row in edges:
    GOT.add_edge(row[0],row[1])
    GOT.edge[row[0]][row[1]]['relation']=row[2]

print('La red tiene ',GOT.number_of_nodes(),'nodos y ',GOT.number_of_edges(),'arcos')


La red tiene  208 nodos y  404 arcos


### 2.A. Encontrar a los 5 personajes más conectados de la red por descendencia y matrimonios. ¿Qué se puede interpretar del rol de estos personajes en la red?

In [2]:
#Matrimonios
matrimonios=[]
for (i,j) in GOT.edges_iter():
    if(GOT.edge[i][j]['relation']=='spouse'):
        matrimonios.append((i,j,GOT.edge[i][j]))
matrimonios_sg=nx.DiGraph(matrimonios)
#Uso in_degree en lugar de degree para que no se me dupliquen los matrimonios porque es una relación recíproca
matrimonios_ordenados=sorted(matrimonios_sg.in_degree_iter(),key=operator.itemgetter(1),reverse=True)
matrimonios_mas=matrimonios_ordenados[0:5]
print('Los 5 personajes con más matrimonios son:',matrimonios_mas)

#Descendencia
descendencia=[]
for (i,j) in GOT.edges_iter():
    if not(GOT.edge[i][j]['relation'] == 'spouse'):
        descendencia.append((i,j,GOT.edge[i][j]))
descendencia_sg=nx.DiGraph(descendencia)
#Uso out_degree para que sólo me queden los hijos del nodo, no sus padres
descendencia_ordenada=sorted(descendencia_sg.out_degree_iter(),key=operator.itemgetter(1),reverse=True)
descendencia_mas=descendencia_ordenada[0:5]

print('Los 5 personajes con más hijos son:', descendencia_mas)

print( 'Estos personajes tienen muchos hijos y matrimonios:',set([d[0] for d in descendencia_mas]).intersection([d[0] for d in matrimonios_mas]))


            



Los 5 personajes con más matrimonios son: [('Jon Arryn', 3), ('Margaery Tyrell', 3), ('Walder Frey', 3), ('Quellon Greyjoy', 3), ('Daenerys Targaryen', 3)]
Los 5 personajes con más hijos son: [('Quellon Greyjoy', 9), ('Oberyn Martell', 8), ('Walder Frey', 7), ('Eddard Stark', 6), ('Bethany Rosby', 5)]
Estos personajes tienen muchos hijos y matrimonios: {'Quellon Greyjoy', 'Walder Frey'}


In [3]:
# Para algunas mediciones convierto la red a una red no dirigida porque no quiero que cuente como un nodo posible que un nodo sea padre de su padre 
#o que el número de matrimonios tenga el doble de peso que el número de hijos.

GOT_un= GOT.to_undirected()

grado=sorted(GOT_un.degree_iter(),key=operator.itemgetter(1),reverse=True)
print('Los 5 personajes con más relaciones familiares son:', grado[0:5])

Los 5 personajes con más relaciones familiares son: [('Quellon Greyjoy', 12), ('Walder Frey', 10), ('Oberyn Martell', 10), ('Eddard Stark', 9), ('Catelyn Stark', 8)]


### Conclusiones: 
Dos de los personajes con más hijos a la vez tienen más matrimonios. Probablemente ellos conectaron varias partes de la red. Sin embargo, la diferencia entre estos conjuntos sugiere que no hay una relación directa entre número de matrimonios e hijos. 

### 2.B. Encontrar la casa más densamente conectada (medida como la densidad del subgrafo que representa a cada casa). ¿Qué se puede interpretar acerca de la estructura de esta casa con respecto a las demás?


In [4]:
#Crea subgrafo por casa
casas=set([v['House'] for (k,v) in GOT_un.node.items() ])
densidad=[]
for casa in casas:
    nodos_casa=[]
    for i in range(len(GOT_un.nodes())):
        if (GOT_un.node[GOT_un.nodes()[i]]['House']== casa):
            nodos_casa.append(GOT_un.nodes()[i])
            #print(nodos_casa)
    SG=GOT_un.subgraph(nodos_casa)
    densidad.append((casa, nx.density(SG)))

densidad_casas=sorted(densidad,key=operator.itemgetter(1),reverse=True)
print('La 3 casas más densamente conectadas en la red no direccionada son:',densidad_casas[0:3])

#subgrafos direccionados
#Crea subgrafo por casa
casas=set([v['House'] for (k,v) in GOT.node.items() ])
densidad=[]
for casa in casas:
    nodos_casa=[]
    for i in range(len(GOT.nodes())):
        if (GOT_un.node[GOT.nodes()[i]]['House']== casa):
            nodos_casa.append(GOT.nodes()[i])
            #print(nodos_casa)
    SG=GOT.subgraph(nodos_casa)
    densidad.append((casa, nx.density(SG)))

densidad_casas=sorted(densidad,key=operator.itemgetter(1),reverse=True)
print('Las 3  casas más densamente conectadas en la  red direccionada son:',densidad_casas[0:3])


    



La 3 casas más densamente conectadas en la red no direccionada son: [('House Bolton', 1.0), ('House Tully', 0.5), ('House Arryn', 0.4)]
Las 3  casas más densamente conectadas en la  red direccionada son: [('House Bolton', 0.5), ('House Arryn', 0.25), ('House Tully', 0.25)]


### 2.C. Encontrar la casa con más conexiones con otras casas. ¿Qué se puede interpretar acerca del rol de esta casa en el sistema social de GOT?
*Utilizo Got_un porque no quiero que me cuente un matrimonio con alguien de otra casa como dos relaciones externas*

In [20]:
num_relaciones_externas={}
for (i,j) in GOT_un.edges_iter():
    if not (GOT_un.node[i]['House']== GOT_un.node[j]['House']):
        num_relaciones_externas[GOT_un.node[i]['House']]= num_relaciones_externas.get(GOT_un.node[i]['House'],0)+1
        num_relaciones_externas[GOT_un.node[j]['House']]= num_relaciones_externas.get(GOT_un.node[j]['House'],0)+1

num_relaciones_externas= [(k,v) for k, v in num_relaciones_externas.items()]
num_relaciones_externas_orden=sorted(num_relaciones_externas,key=operator.itemgetter(1),reverse=True)        

print(num_relaciones_externas_orden)      

[('House Lannister', 47), ('House Stark', 29), ('House Frey', 27), ('House Tyrell', 22), ('House Greyjoy', 18), ('House Tully', 14), ('House Baratheon', 14), ('House Targaryen', 14), ('House Martell', 13), ('House Darry', 8), ('House Marbrand', 8), ('House Royce', 7), ('House Crakehall', 6), ('House Rosby', 6), ('House Sunderly', 6), ('House Redwyne', 6), ('House Hightower', 5), ('House Blackwood', 5), ('House Harlaw', 5), ('House Arryn', 4), ('House Whent', 4), ('House Stonetree', 4), ('House Lefford', 4), ('House Dorne', 4), ('House Uller', 4), ('House Karstark', 4), ('House Norridge', 4), ('House Swyft', 3), ('House Brax', 3), ('House Prester', 3), ('House Serry', 3), ('House Moryn', 3), ('House Florent', 2), ('House Bolton', 2), ('House Piper', 2), ('House Stackspear', 2), ('House Webber', 2), ('House Flint', 2), ('House Glover', 2), ('House Locke', 2), ('House Waynwood', 1), ('House Baelish', 1), ('House Ironmaker', 1), ('House Farman', 1), ('House Jast', 1), ('House Hayford', 1),

### 2.D. Encontrar el personaje de cada casa principal que en promedio se encuentra más cerca a los personajes de la red. ¿Cuál es una posible explicación de la posición de estos personajes?
*se usa la red no dirigida porque en general para llegar de un nodo a otro no es relevante la direccionalidad de estas relaciones*

In [23]:
cercania=[]
for casa in casas:
    cercania_casa=[]
    for i in range(len(GOT_un.nodes())):
        if (GOT.node[GOT_un.nodes()[i]]['House']== casa):
            cercania_casa.append((GOT_un.nodes()[i],nx.closeness_centrality(GOT_un,GOT_un.nodes()[i])))
            cercania_casa=sorted(cercania_casa,key=operator.itemgetter(1),reverse=True)
    cercania.append((casa,cercania_casa ))
cercania_orden= [(d[0],sorted(d[1],key=operator.itemgetter(1),reverse=True)) for d in cercania]
mas_cercano_por_casa= [(d[0],d[1][0]) for d in cercania_orden]
print(mas_cercano_por_casa)


[('House Waynwood', ('Elys Waynwood', 0.08138415698410725)), ('House Norrey', ('Arra Norrey', 0.07901155083417724)), ('House Martell', ('Elia Martell', 0.12292581810846437)), ('House Ryswell', ('Robyn Ryswell', 0.07281192381776418)), ('House Oldstones', ('Jenny of Oldstones', 0.09858196382947865)), ('House Dorne', ('Princess of Dorne', 0.11015739442106905)), ('House Bolton', ('Ramsay Bolton', 0.14631016396971466)), ('House Blackbar', ('Leo Blackbar', 0.06932357342779416)), ('House Stackspear', ('Alys Stackspear', 0.12554703040636545)), ('House Sunderly', ('Lady of House Sunderly', 0.05049088359046283)), ('House Uller', ('Ellaria Sand', 0.08972357401610982)), ('House Arryn', ('Jon Arryn', 0.11065713632706223)), ('House Karstark', ('Alys Karstark', 0.09459499243914515)), ('House Tysha', ('Tysha', 0.14193180494817706)), ('House Fossoway', ('Leonette Fossoway', 0.09464742868772562)), ('House Ironmaker', ('Erik Ironmaker', 0.02608695652173913)), ('House Prester', ('Marla Prester', 0.1428819

### 2.E. Encontrar a los 5 personajes que se encuentran más veces dentro del camino más corto entre todos los nodos de la red. ¿Qué puede suceder con la topología de la red si estos personajes son asesinados?
*utilizo la red no dirigida porque en la inormación puede fluir sin importar la dirección de la relación, es decir, es posible que fluya de hijos a padres.*

In [311]:
betweenness=nx.betweenness_centrality(GOT_un)
betweenness= [(k,v) for k, v in betweenness.items()]
betweenness_ordenadas=sorted(betweenness,key=operator.itemgetter(1),reverse=True)     
print(betweenness_ordenadas[0:5])



[('Eddard Stark', 0.3248845805680062), ('Sansa Stark', 0.28918280208446345), ('Tyrion Lannister', 0.2634718017568876), ('Tywin Lannister', 0.23781225082703206), ('Joanna Lannister', 0.2221999191390007)]


### 2.F. Encontrar la probabilidad de que los padres de un personaje sean esposos. ¿Qué puede inferir de las relaciones familiares de los personajes de GOT?

In [25]:
familia_nuclear=[]

for i in GOT.nodes_iter():
    parents= {}
    for j in GOT.nodes_iter():
        if(GOT.has_edge(j,i)):
            if not (GOT.edge[j][i]['relation']=='spouse'):
                    #print(i,' is child of ',j,GOT.edge[j][i])
                    parents[GOT.edge[j][i]['relation']] = j
                    
          
    if('mother' in parents and 'father' in parents):
        #print(i, parents)
        if(GOT.has_edge(parents['mother'], parents['father'])):
            if GOT.edge[parents['mother']][parents['father']]['relation'] ==  'spouse':
                familia_nuclear.append(1)
        else:
            familia_nuclear.append(0)
                    
print('La probabilidad de que los padres de un personaje sean esposos es: ', sum(familia_nuclear)/len(familia_nuclear))


La probabilidad de que los padres de un personaje sean esposos es:  0.9122807017543859


### 2.G. Encontrar a los 5 personajes más conectados y cuyos vecinos tienen mayor grado. ¿Qué puede inferir de la influencia de estos personajes en la red?
*De nuevo, utilizo  la red no dirigida porque no quiero que me cuente el matrimonio como dos relaciones*

In [26]:
eigenvector_centrality= nx.eigenvector_centrality_numpy(GOT_un)
eigenvector_centrality= [(k,v) for k, v in eigenvector_centrality.items()]
eigenvector_centrality_ordenado= sorted(eigenvector_centrality,key=operator.itemgetter(1),reverse=True)
print('Los 5 personajes con mayor centralidad eigenvector son: ', eigenvector_centrality_ordenado[0:5])


Los 5 personajes con mayor centralidad eigenvector son:  [('Tywin Lannister', 0.29286328772259573), ('Cersei Lannister', 0.2685257630832046), ('Joanna Lannister', 0.24477869576073166), ('Jeyne Marbrand', 0.23985724797654798), ('Tytos Lannister', 0.23985724797654792)]


### Visualice la red resaltando a los personajes encontrados en los literales anteriores con distintos colores según la medida de centralidad. Especifique los casos en que un personaje aparezca en varias medidas.